---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [49]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    university_towns = pd.read_table("university_towns.txt",header=None).rename(columns={0:'RegionName'})
    university_towns['State']=0
    university_towns = university_towns[['State','RegionName']]
    for index,row in university_towns.iterrows():
        if "[edit]" in row['RegionName']:
            state_name = row['RegionName'].split('[')[0]
            university_towns.loc[index,'State'] = state_name
            university_towns.loc[index,'RegionName'] = row['RegionName'].split('[')[0]
        else:
            university_towns.loc[index,'State'] = state_name
            university_towns.loc[index,'RegionName'] = row['RegionName'].split(' (')[0]
    university_towns = university_towns[(university_towns['State']!=university_towns['RegionName'])].reset_index(drop=True)
    return university_towns

In [60]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp.drop(['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 7'], axis=1,inplace=True)
    gdp.rename(columns={'Unnamed: 4':'time','Unnamed: 6':'GDP'}, inplace=True)
    gdp.drop(gdp.head(216).index,axis=0,inplace=True)
    gdp['recession1'] = gdp['GDP'].diff(1).shift(-1)
    gdp['recession2'] = gdp['recession1'].shift(-1)
    gdp['end_recession1'] = gdp['GDP'].diff(-1).shift(1)
    gdp['end_recession2'] = gdp['end_recession1'].shift(1)
    recession_start_data = gdp[(gdp['recession1']<0) & (gdp['recession2']<0) & (gdp['end_recession1']>0)]
    recession_start_data['data_num'] = recession_start_data.index
    recession_start_data['data_num_delta'] = recession_start_data['data_num'].diff(1)
    recession_start_data['if_near'] = recession_start_data['data_num_delta'].map(lambda x:1 if x<=2 else 0)
    return recession_start_data[recession_start_data['if_near']==0]['time'].tolist()[0]

'2008q3'

In [62]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp.drop(['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 7'], axis=1,inplace=True)
    gdp.rename(columns={'Unnamed: 4':'time','Unnamed: 6':'GDP'}, inplace=True)
    gdp.drop(gdp.head(216).index,axis=0,inplace=True)
    gdp['recession1'] = gdp['GDP'].diff(1).shift(-1)
    gdp['recession2'] = gdp['recession1'].shift(-1)
    recession_start_data = gdp[(gdp['recession1']<0) & (gdp['recession2']<0)]
    recession_start_data['data_num'] = recession_start_data.index
    recession_start_data['data_num_delta'] = recession_start_data['data_num'].diff(1)
    recession_start_data['if_near'] = recession_start_data['data_num_delta'].map(lambda x:1 if x<=2 else 0)
    recession_start_data['if_start'] = 1
    recession_start_data = recession_start_data[recession_start_data['if_near']==0]
    gdp['end_recession1'] = gdp['GDP'].diff(-1).shift(1)
    gdp['end_recession2'] = gdp['end_recession1'].shift(1)
    recession_end_data = gdp[(gdp['end_recession1']<0) & (gdp['end_recession2']<0)]
    recession_end_data['data_num'] = recession_end_data.index
    recession_end_data['data_num_delta'] = recession_end_data['data_num'].diff(1)
    recession_end_data['if_near'] = recession_end_data['data_num_delta'].map(lambda x:1 if x<=2 else 0)
    recession_end_data['if_start'] = 0    
    recession_end_data = recession_end_data[recession_end_data['if_near']==0]
    recession = pd.concat([recession_start_data,recession_end_data]).sort_index()
    recession['recession_foward'] = recession['if_start'].diff(1).shift(-1)
    recession['recession_backward'] = recession['if_start'].diff(-1).shift(1)
    return recession[recession['recession_backward']==1]['time'].tolist()[0]

In [6]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    gdp = pd.read_excel('gdplev.xls',skiprows=7)
    gdp.drop(['Unnamed: 0', 'Unnamed: 1','Unnamed: 2','Unnamed: 3','Unnamed: 5','Unnamed: 7'], axis=1,inplace=True)
    gdp.rename(columns={'Unnamed: 4':'time','Unnamed: 6':'GDP'}, inplace=True)
    gdp.drop(gdp.head(216).index,axis=0,inplace=True)
    gdp = gdp.set_index('time')
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_period = gdp.loc[recession_start:recession_end]
    return recession_period['GDP'].argmin()

In [48]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    import datetime as dt
    price = pd.read_csv('City_Zhvi_AllHomes.csv')
    price.replace({'State':states}, inplace = True)
    price = price.set_index(['State','RegionName'])
    dates = pd.DataFrame(index=pd.date_range('01-2000', '09-2016', freq='M'))
    dates_list = dates.index.to_series().apply(lambda x: dt.datetime.strftime(x,'%Y-%m')).tolist()
    price = price[dates_list]
    price = price.groupby(pd.PeriodIndex(price.columns, freq='Q'),axis=1).mean().rename(columns=lambda c: str(c).lower())    
    return price

In [66]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    price = convert_housing_data_to_quarters()
    if get_recession_start().split('q')[1]=='1':
        first = str(int(get_recession_start().split('q')[0])-1)
        second = 'q4'
        start_year = first + second
    elif get_recession_start().split('q')[1]=='2':
        first = get_recession_start().split('q')[0]
        second = 'q1'
        start_year = first + second
    elif get_recession_start().split('q')[1]=='3':
        first = get_recession_start().split('q')[0]
        second = 'q2'
        start_year = first + second
    else:
        first = get_recession_start().split('q')[0]
        second = 'q3'
        start_year = first + second
    bottom = get_recession_bottom()
    university_towns = get_list_of_university_towns()
    price['ratio'] = price[start_year]/price[bottom]
    university_towns = university_towns.set_index(['State','RegionName'])
    university_towns_price = pd.merge(price,university_towns,how='right',left_index=True, right_index=True)['ratio']
    university_towns_price = university_towns_price.dropna()
    not_university_towns = set(price.index) - set(university_towns_price.index)
    not_university_towns_price = price.loc[list(not_university_towns)]['ratio'].dropna()
    statistic,p_value = tuple(ttest_ind(university_towns_price,not_university_towns_price))
    different = p_value<0.01
    if statistic<0:
        better = 'university town'
    else:
        better = 'non-university town'
    return (different,p_value,better)